The dataset for this colab is available: [Training and validation set](https://drive.google.com/file/d/1Ji3oeXia4H4dd4bWpxkgTRfECS6Avn1d/view?usp=sharing) and [Test dataset](https://drive.google.com/file/d/1FWyWSnHNb28bit3mj2xrW5KpcvPKglGo/view?usp=sharing). 

The purpose of this study was to forecast the auction sale price of a specific piece of heavy equipment (a bulldozer) based on its usage, equipment type, and configuration. The information comes from auction result postings and covers use and equipment settings.
The RMSLE (root mean squared log error) between the actual and forecast auction prices is the competition's evaluation metric.

The data page contains links to sample submission files. Files submitted should be formatted as follows:

A newly created` header  `SalesID,SalePrice`;
has two columns:

`SalesID`: The validation set's SalesID in sorted order;
`SalePrice`: Your estimated sale price.

# [Root Mean Squared Log Error (RMSLE)](https://www.kaggle.com/code/carlolepelaars/understanding-the-metric-rmsle)

The formula for RMSLE is as follows:

$RMSLE=\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(p_i +1)-log(a_i +1))^2}$

where:

$n$  is the total number of observations in the (public/private) data set,

$p_i$  is the prediction of target, and

$a_i$  is the actual target for  $i$ .

$log(x)$  is the natural logarithm of  $x  ( log_e(x)$ .

#Prepare tools

##Random Forest Regression

Every decision tree has a significant variance, but when we mix all of them in parallel, the final variance is low since each decision tree is perfectly trained on that specific sample data, and so the outcome is dependent on numerous decision trees rather than one. In the case of a classification problem, the majority voting classifier is used to determine the final output. The final output of a regression problem is the mean of all the outputs. Aggregation is the name given to this section.

Random Forest is an ensemble technique that can handle both regression and classification tasks by combining many decision trees and a technique known as Bootstrap and Aggregation, or bagging. The core idea is to use numerous decision trees to determine the final output rather than depending on individual decision trees.

In [ ]:
# import exploratory data analysis and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# models from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# model evaluations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#Inspect data

#Q1
Import the training and validation dataset `TrainAndValid.csv` with `low_memory=False`and priint first 5 rows of the dataset. What is the `auctionerrID` for `SalesID=1139246`? 



In [ ]:
data = pd.read_csv('/content/sample_data/TrainAndValid.csv', low_memory=False)
data[data['SalesID'] == 1139246]

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#Q2
How many non-null values are there in the state column?

**412698**



In [ ]:
# data['state'].isna().sum()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              392562 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  147504 non-null  float64
 8   UsageBand                 73670 non-null   object 
 9   saledate                  412698 non-null  object 
 10  fiModelDesc               412698 non-null  object 
 11  fiBaseModel               412698 non-null  object 
 12  fiSecondaryDesc           271971 non-null  object 
 13  fiModelSeries             58667 non-null   o

#Q3
Which of the following columns have missing values?

In [ ]:
print('MachineHoursCurrentMeterdata: ', data['MachineHoursCurrentMeter'].isna().sum())
print('fiModelDesc: ', data['fiModelDesc'].isna().sum())
print('fiBaseModel: ', data['fiBaseModel'].isna().sum())
print('ProductSize: ', data['ProductSize'].isna().sum())

MachineHoursCurrentMeterdata:  265194
fiModelDesc:  0
fiBaseModel:  0
ProductSize:  216605


#Q4
How many columns are there in the TrainAndValid dataset?

In [ ]:
data.shape

(412698, 53)

#Q5
Can you sort the dataframe in ascending order of saledate with inplace=True? Which of the following options represent the saledate of the salesID 1646770 ?

**1989-01-17**

1989-01-31

1992-03-25

2022-11-11

In [ ]:
data.sort_values(by='saledate', inplace=True)

In [ ]:
data[data['SalesID'] == 1646770].saledate

205615    1/17/1989 0:00
Name: saledate, dtype: object

#Q6
Make a copy of the dataset. What is the year of making (YearMade) of the product with MachineID 1194089?

**1980**

1974

1982

1947

In [ ]:
data_copy = data.copy()

In [ ]:
data[data['MachineID'] == 1194089].YearMade

274835    1980
Name: YearMade, dtype: int64

#Q
Import data again but this time parse dates

In [ ]:
df = pd.read_csv("/content/sample_data/TrainAndValid.csv",
                 low_memory=False,
                 parse_dates=["saledate"])

In [ ]:
df.shape

(412698, 53)

#Q7
Separate the saledate in saleYear,saleMonth,saleDay,saleDayOfWeek and saleDayOfYear and saledate column. Which numeric columns from the following options have null values?

saleDayOfWeek

SalePrice

**auctioneerID**

**MachineHoursCurrentMeter**

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy['saleYear'] = df_copy.saledate.dt.year
df_copy['saleMonth'] = df_copy.saledate.dt.month
df_copy['saleDay'] = df_copy.saledate.dt.day
df_copy['saleDayOfWeek'] = df_copy.saledate.dt.day_of_week
df_copy['saleDayOfYear'] = df_copy.saledate.dt.day_of_year
df_copy.drop('saledate', axis=1, inplace=True)
df_copy.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfWeek,saleDayOfYear
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,521D,...,NaN,NaN,NaN,Standard,Conventional,2006,11,16,3,320
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,950FII,...,NaN,NaN,NaN,Standard,Conventional,2004,3,26,4,86
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,226,...,NaN,NaN,NaN,NaN,NaN,2004,2,26,3,57
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,NaN,NaN,NaN,NaN,NaN,2011,5,19,3,139
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,NaN,NaN,NaN,NaN,NaN,2009,7,23,3,204


In [ ]:
num_columns = df_copy.describe().columns
print(num_columns)
df_copy[num_columns].isna().sum()

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'saleYear',
       'saleMonth', 'saleDay', 'saleDayOfWeek', 'saleDayOfYear'],
      dtype='object')


SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
saleYear                         0
saleMonth                        0
saleDay                          0
saleDayOfWeek                    0
saleDayOfYear                    0
dtype: int64

#Q8
How many values are missing in the column auctioneerID_is_missing?

**20136**

21152

41512

None of these

In [ ]:
df_copy['auctioneerID'].isna().sum()

20136

#Q9
Convert the state and Usageband to numbers. What is the datatype?


[Hint: use pd.Categorical(df_tmp["feature_name"]).codes]

float

**int**

none of the above

In [ ]:
pd.Categorical(data['state']).codes

array([21, 37, 37, ..., 42, 43,  4], dtype=int8)

#Q10
Add binary columns (with label+"_is_missing") to indicate whether sample had missing value.Turn all categories into numbers.All null values will be turned into "-1" so add +1 to all numbers. What is the value of Differential_Type_is_missing for SalesID=1646770?

**True**

False

In [ ]:
cat_columns = [column for column in df_copy.columns if column not in num_columns]
cat_columns

['UsageBand',
 'fiModelDesc',
 'fiBaseModel',
 'fiSecondaryDesc',
 'fiModelSeries',
 'fiModelDescriptor',
 'ProductSize',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc',
 'Drive_System',
 'Enclosure',
 'Forks',
 'Pad_Type',
 'Ride_Control',
 'Stick',
 'Transmission',
 'Turbocharged',
 'Blade_Extension',
 'Blade_Width',
 'Enclosure_Type',
 'Engine_Horsepower',
 'Hydraulics',
 'Pushblock',
 'Ripper',
 'Scarifier',
 'Tip_Control',
 'Tire_Size',
 'Coupler',
 'Coupler_System',
 'Grouser_Tracks',
 'Hydraulics_Flow',
 'Track_Type',
 'Undercarriage_Pad_Width',
 'Stick_Length',
 'Thumb',
 'Pattern_Changer',
 'Grouser_Type',
 'Backhoe_Mounting',
 'Blade_Type',
 'Travel_Controls',
 'Differential_Type',
 'Steering_Controls']

In [ ]:
# for column in cat_columns:
#   print(pd.Categorical(df_copy[column]).codes)

for label, content in df_copy.items():
  if label in num_columns:
    if(pd.isnull(content).any()):
      df_copy[label+'_is_missing'] = pd.isnull(content)
      df_copy[label] = content.fillna(content.median())

  if label in cat_columns:
    if(pd.isnull(content).any()):
      df_copy[label+'_is_missing'] = pd.isnull(content)
    df_copy[label] = pd.Categorical(content).codes + 1
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 97 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   SalesID                              412698 non-null  int64  
 1   SalePrice                            412698 non-null  float64
 2   MachineID                            412698 non-null  int64  
 3   ModelID                              412698 non-null  int64  
 4   datasource                           412698 non-null  int64  
 5   auctioneerID                         412698 non-null  float64
 6   YearMade                             412698 non-null  int64  
 7   MachineHoursCurrentMeter             412698 non-null  float64
 8   UsageBand                            412698 non-null  int8   
 9   fiModelDesc                          412698 non-null  int16  
 10  fiBaseModel                          412698 non-null  int16  
 11  fiSecondaryDe

In [ ]:
# for column in num_columns:
#   if df_copy[column].isnull().any():
#     # print(column, df_copy[column].isnull())
#     df_copy[column+'_is_missing'] = df_copy[column].isnull()
#     df_copy[column] = df_copy[column].fillna(df_copy[column].mean())   #### to fill median values in null columns

# for column in cat_columns:
#   if df_copy[column].isnull().any():
#     # print(column, df_copy[column].isnull())
#     df_copy[column+'_is_missing'] = df_copy[column].isnull()
#     df_copy[column] = pd.Categorical(df_copy[column]).codes + 1

In [ ]:
# df_copy.head()
df_copy[df_copy['SalesID'] == 1646770].Differential_Type_is_missing

205615    True
Name: Differential_Type_is_missing, dtype: bool

In [ ]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 97 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   SalesID                              412698 non-null  int64  
 1   SalePrice                            412698 non-null  float64
 2   MachineID                            412698 non-null  int64  
 3   ModelID                              412698 non-null  int64  
 4   datasource                           412698 non-null  int64  
 5   auctioneerID                         412698 non-null  float64
 6   YearMade                             412698 non-null  int64  
 7   MachineHoursCurrentMeter             412698 non-null  float64
 8   UsageBand                            412698 non-null  int8   
 9   fiModelDesc                          412698 non-null  int16  
 10  fiBaseModel                          412698 non-null  int16  
 11  fiSecondaryDe

#Q11
Is there any null value in the columns?

True

**False**

In [ ]:
df_copy.isna().sum().sort_values(ascending=False)

SalesID                         0
Travel_Controls                 0
Turbocharged_is_missing         0
Transmission_is_missing         0
Stick_is_missing                0
                               ..
Engine_Horsepower               0
Enclosure_Type                  0
Blade_Width                     0
Blade_Extension                 0
Steering_Controls_is_missing    0
Length: 97, dtype: int64

#Q12
Split data into training and validation sets. Data from 2011 will be used for validation. What is the length of the training dataset?

**(377501, 97)**

In [ ]:
df_copy.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Undercarriage_Pad_Width_is_missing,Stick_Length_is_missing,Thumb_is_missing,Pattern_Changer_is_missing,Grouser_Type_is_missing,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,2,963,...,True,True,True,True,True,True,True,True,False,False
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,2,1745,...,True,True,True,True,True,True,True,True,False,False
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,1,336,...,True,True,True,True,True,True,True,True,True,True
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,1,3716,...,True,True,True,True,True,True,True,True,True,True
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,3,4261,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
df_val = df_copy[df_copy['saleYear'] == 2011]
df_train = df_copy[df_copy['saleYear'] != 2011]
df_train.shape, df_val.shape
len(df_train), len(df_val)

(377501, 35197)

#Q13
Differentiate the training dataset into two different dataframes X_train and y_train where y_train is the SalePrice feature and X_train has the remaining features. Differentiate the validation dataframe into X_val and y_val in same manner.

Which of the following options represent the shapes of X_train and y_val?

**(377501, 102), (35197,)**  

(377501,), (35197, 102)

None of these

In [ ]:
X_train, y_train = df_train.drop('SalePrice', axis=1), df_train['SalePrice']
X_val, y_val = df_val.drop('SalePrice', axis=1), df_val['SalePrice']
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(377501, 96) (377501,) (35197, 96) (35197,)


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377501 entries, 0 to 412697
Data columns (total 96 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   SalesID                              377501 non-null  int64  
 1   MachineID                            377501 non-null  int64  
 2   ModelID                              377501 non-null  int64  
 3   datasource                           377501 non-null  int64  
 4   auctioneerID                         377501 non-null  float64
 5   YearMade                             377501 non-null  int64  
 6   MachineHoursCurrentMeter             377501 non-null  float64
 7   UsageBand                            377501 non-null  int8   
 8   fiModelDesc                          377501 non-null  int16  
 9   fiBaseModel                          377501 non-null  int16  
 10  fiSecondaryDesc                      377501 non-null  int16  
 11  fiModelSeries

#Q14
Fit a RandomForestRegressor 'model' with n_jobs=-1, random state =42 and max_samples=10000 with X_tran and y_train.RMSLE calculates root mean squared log error between predictions (y_pred) and true labels (y). Create function to evaluate and return the values for aforementioned model using different metrics, such as MAE, RMSLE and R^2R 
2
  for training set ,i.e. (y_train, train_preds) and validation set ,i.e. (y_val, val_preds). What is the R^2R 
2
  value for validation dataset (upto 4 decimal points)?

In [ ]:
model = RandomForestRegressor(n_jobs=1, random_state=42, max_samples=10000)
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.8615923789765196

In [ ]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_true, y_pred):
  return np.sqrt(mean_squared_log_error(y_true, y_pred))

def show_scores(model, X_train, y_train, X_val, y_val):
  y_train_pred = model.predict(X_train)
  y_val_pred = model.predict(X_val)

  scores = {"training MAE" : mean_absolute_error(y_train, y_train_pred), 
            "validation MAE" : mean_absolute_error(y_val, y_val_pred),
            "training RMSLE" : rmsle(y_train, y_train_pred),
            "validation RMSLE" : rmsle(y_val, y_val_pred),
            "training R2" : r2_score(y_train, y_train_pred),
            "validation R2" : r2_score(y_val, y_val_pred)
  }
  return scores

In [ ]:
scores = show_scores(model, X_train, y_train, X_val, y_val)
print(scores)
print(scores['validation R2'])

{'training MAE': 5524.837731131837, 'validation MAE': 6653.686115009802, 'training RMSLE': 0.2573101880684329, 'validation RMSLE': 0.27914137258136434, 'training R2': 0.8615923789765196, 'validation R2': 0.8297130950552913}
0.8297130950552913


#Q15
### Hyperparameters tuning with RandomizedSearchCV
Instantiate RandomiedSearchCV model with 5 fold cross validation, verbose=True, 10 iterations, n_jobs=-1, random state=42 and rf_grid as follows:

             `"n_estimators": [30, 40, 50],
       "max_depth": [None, 3, 5, 10],
       "min_samples_split": [10, 14, 18]
       "min_samples_leaf": [1, 2, 3],
       "max_features": [0.5, 1, "sqrt", "auto"],
       "max_samples": [10000]`

Find the best model parameters and calculate  R^2R 
2
   for training dataset (upto 4 decimal points).

In [ ]:
param_grid = {"n_estimators": [30, 40, 50],
   "max_depth": [None, 3, 5, 10],
   "min_samples_split": [10, 14, 18],
   "min_samples_leaf": [1, 2, 3],
   "max_features": [0.5, 1, "sqrt", "auto"],
   "max_samples": [10000]
}
random_srch = RandomizedSearchCV(model, param_grid, cv=5, n_iter=10, verbose=True, n_jobs=-1, random_state=42)
random_srch.fit(X_train, y_train)
random_srch.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomForestRegressor(max_depth=10, max_samples=10000, min_samples_leaf=3,
                      min_samples_split=10, n_estimators=40, n_jobs=1,
                      random_state=42)

In [ ]:
random_srch.best_params_

{'n_estimators': 40,
 'min_samples_split': 10,
 'min_samples_leaf': 3,
 'max_samples': 10000,
 'max_features': 'auto',
 'max_depth': 10}

In [ ]:
y_train_pred = random_srch.best_estimator_.predict(X_train)
score = r2_score(y_train, y_train_pred)
print('R2 score of training dataset: ', score)
# print(show_scores(random_srch, X_train, y_train, X_val, y_val))

R2 score of training dataset:  0.8156482312288763


#Q16
###Hyperparameters tuning with GridSearchCV
Instantiate GridSearchCV model with 5 fold cross validation, verbose=True, 10 iterations, random state=42, n_jobs=-1 and param_grid as follows:

      `"n_estimators": [170, 200],
       "min_samples_split": [4, 5],
       "min_samples_leaf": [2],
       "max_features": [0.4, 0.45, 0.5],
       "max_samples": [10000]`

Find the best model parameters and calculate  R^2R 
2
   for training dataset (upto 4 decimal points).

In [ ]:
# param_grid = {"n_estimators": [170, 200],
#    "min_samples_split": [4, 5],
#    "min_samples_leaf": [2],
#    "max_features": [0.4, 0.45, 0.5],
#    "max_samples": [10000]    
# }
# grid_srch = GridSearchCV(model, param_grid=param_grid, cv=5, verbose=True, n_jobs=-1)
# grid_srch.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(max_samples=10000, n_jobs=1,
                                             random_state=42),
             n_jobs=-1,
             param_grid={'max_features': [0.4, 0.45, 0.5],
                         'max_samples': [10000], 'min_samples_leaf': [2],
                         'min_samples_split': [4, 5],
                         'n_estimators': [170, 200]},
             verbose=True)

In [ ]:
# grid_srch.best_params_

{'max_features': 0.5,
 'max_samples': 10000,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 200}

In [ ]:
# grid_srch.best_estimator_

RandomForestRegressor(max_features=0.5, max_samples=10000, min_samples_leaf=2,
                      min_samples_split=4, n_estimators=200, n_jobs=1,
                      random_state=42)

In [ ]:
# y_train_pred = grid_srch.best_estimator_.predict(X_train)
# print(r2_score(y_train, y_train_pred))

0.8536129620756233


#Q17
###Make predictions on test data
Import the test dataset Test.csv with low_memory=False and parse the saledate. Print the first 5 rows of test dataset. How many null values are there in the Hydraulics column of the test dataset df_test?

**2142**

In [ ]:
test_df = pd.read_csv('/content/sample_data/Test.csv', low_memory=False, parse_dates=['saledate'])

In [ ]:
test_df.shape

(12457, 52)

In [ ]:
test_df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829,1006309,3168,121,3,1999,3688.0,Low,2012-05-03,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844,1022817,7271,121,3,1000,28555.0,High,2012-05-10,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847,1031560,22805,121,3,2004,6038.0,Medium,2012-05-10,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848,56204,1269,121,3,2006,8940.0,High,2012-05-10,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863,1053887,22312,121,3,2005,2286.0,Low,2012-05-10,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [ ]:
print(test_df['Hydraulics'].isna().sum())
test_df.isna().sum()

2142


SalesID                         0
MachineID                       0
ModelID                         0
datasource                      0
auctioneerID                    0
YearMade                        0
MachineHoursCurrentMeter    10328
UsageBand                   10623
saledate                        0
fiModelDesc                     0
fiBaseModel                     0
fiSecondaryDesc              3975
fiModelSeries               10451
fiModelDescriptor            9433
ProductSize                  6409
fiProductClassDesc              0
state                           0
ProductGroup                    0
ProductGroupDesc                0
Drive_System                 9698
Enclosure                       2
Forks                        6149
Pad_Type                    10349
Ride_Control                 8216
Stick                       10349
Transmission                 7639
Turbocharged                10349
Blade_Extension             11806
Blade_Width                 11806
Enclosure_Type

#Q18
###Preprocessing test data (getting it into same format as training dataset)
Preprocess test data using the same stepsused in training dataset, i.e. separate saledate into saleYear,saleMonth, saleDay, saleDayOfWeek, saleDayOfYear and drop the original saledate feature. Add binary column which tells us if data was missing or not, fill numeric values with median, turn categorical variables into numbers and fill missing, add binart column to indicate whether sample had missing value and turn missing categories into number and add +1. Calculate the difference between sets of X_train and df_test columns and mark it from the given options.

{'auctioneerID_is_missing'}

{'Backhoe_Mounting_is_missing'}

{'Differential_Type_is_missing'}

None of these

In [ ]:
test_df['saleYear'] = test_df['saledate'].dt.year
test_df['saleMonth'] = test_df['saledate'].dt.month
test_df['saleDay'] = test_df['saledate'].dt.day
test_df['saleDayOfWeek'] = test_df['saledate'].dt.day_of_week
test_df['saleDayOfYear'] = test_df['saledate'].dt.day_of_year
test_df.drop('saledate', axis=1, inplace=True)
test_df.shape

(12457, 56)

In [ ]:
num_cols = test_df.describe().columns
cat_cols = [column for column in test_df.columns if column not in num_cols]

for label, content in test_df.items():
  if label in num_cols:
    if(pd.isnull(content).any()):
      test_df[label+'_is_missing'] = pd.isnull(content)
      test_df[label] = content.fillna(content.median())
  if label in cat_cols:
    if(pd.isnull(content).any()):
      test_df[label+'_is_missing'] = pd.isnull(content)
    test_df[label] = pd.Categorical(content).codes + 1

test_df.shape

(12457, 95)

In [ ]:
len(X_train.columns) - len(test_df.columns)

1

In [ ]:
missing = [col for col in X_train.columns if col not in test_df.columns]
print(missing)

['auctioneerID_is_missing']
